In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from IPython.core.display import HTML

# Scraping Loop

In [ ]:
base_url = 'https://ryman.com/events/list/?tribe_event_display=list&tribe_paged='
#empty lists taht will be added to throughout the loop for each page
#will eventually be the columns of the dataframe
running_performers = []
running_dates = []
running_times = []
running_open = []
running_price = []
for j in range(1,6):
    #set up url
    page = str(j)
    #current page url
    url  = base_url + page
    #read url
    response = requests.get(url)
    #make soup
    soup = bs(response.text)
    #get performers
    acts = soup.findAll('h2', attrs = {'class':"tribe-events-list-event-title"})
    performer_list = [i.find('a').get('title') for i in acts]
    #get date
    dates = soup.findAll('div', attrs = {'class':"tribe-event-schedule-details"})
    date_info_list = [i.find('p').text for i in dates]
    ###split dates into date and time
    datelist = []
    timelist  = []
    for x in date_info_list:
        split = x.split(' at ')
        datelist.append(split[0])
        timelist.append(split[1])
           
   
    #get price
    price_links = soup.findAll('h2', attrs = {'class':"tribe-events-list-event-title"})
    
    new_urls = [x.find('a').get('href') for x in price_links]
        
    running_price_list = []
    for newlink in new_urls:
        inner_response = requests.get(newlink)
        temp_soup = bs(inner_response.text)
    
        price_html = temp_soup.find('p', attrs = {'class':'theprices'})
        if price_html == None:
            running_price_list.append('CANCELED')
        else:
            running_price_list.append(price_html.text)
    #get opener
    openers_list = []
    boxes = soup.findAll('div', attrs = {'class',"tribe-beside-image"})
    for x in boxes:
        curr = x.findAll('span')
        lenc = len(curr)
        if lenc == 0:
            openers_list.append('no opener')
        elif lenc == 1:
            if curr[0].text == '2nd Show Added!':
                openers_list.append('no opener')
            else:
                openers_list.append(curr[0].text)
        else:
            openers_list.append(curr[1].text)
    #concat on to running lists
    running_performers += performer_list
    running_dates += datelist
    running_times += timelist
    running_open += openers_list
    running_price += running_price_list
#make data frame outside of loop
ryman_events = pd.DataFrame()
ryman_events['event'] = running_performers
ryman_events['details'] = running_open
ryman_events['date'] = running_dates
ryman_events['time'] = running_times
ryman_events['price'] = running_price


In [ ]:
ryman_events

# Cleaning up table columns

#### Price column

In [ ]:
new_price_col = []
for price in ryman_events.price:
    if price[0] != '$':
        if price[0] == "C": 
            new_price_col.append('Canceled')
        else:
            new_price_col.append('Tickets Available for Purchase Elsewhere')
    else:
        price_split = price.split('/')
        if price_split[-1].find('Groups') == -1:
            for index, value in enumerate(price_split):
                price_split[index] = float(value.strip(' ').strip('$'))
            avg_price = sum(price_split) / float(len(price_split))
            new_price_col.append("%.2f" % avg_price)
        else: #case where there is groups on the end of price
            price_split[-1] = price_split[-1].strip().strip('(Groups)').strip('$').strip()
            for index, value in enumerate(price_split):
                price_split[index] = float(value.strip(' ').strip('$'))
                                           
            avg_price = sum(price_split) / float(len(price_split))
            new_price_col.append("%.2f" % avg_price)
ryman_events['price'] = new_price_col
ryman_events = ryman_events.rename(columns = {'price':'avg_ticket_price'})

In [ ]:
ryman_events

# Graph prices

In [ ]:
noncancel_events = ryman_events.loc[~(ryman_events['avg_ticket_price'] == 'Canceled')]

In [ ]:
noncancel_events = noncancel_events.loc[~(noncancel_events['avg_ticket_price'] == 
                                          'Tickets Available for Purchase Elsewhere')]

In [ ]:
noncancel_events['avg_ticket_price'] = noncancel_events['avg_ticket_price'].astype('float')

In [ ]:
noncancel_events['date'] = pd.to_datetime(noncancel_events['date'])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
fig, ax = plt.subplots(figsize = (12,6))
plt.plot(noncancel_events.date,noncancel_events.avg_ticket_price)
plt.xlabel('date')
plt.ylabel('price USD')
plt.suptitle('avg ticket price per show')